In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

In [3]:
my_run = BuildStockQuery(db_name='euss-tests',
                        table_name='res_test_03_2018_10k_20220607',
                        workgroup='eulp',
                        buildstock_type='resstock',
                        skip_reports=True)

INFO:buildstock_query.query_core:Loading res_test_03_2018_10k_20220607 ...
INFO:buildstock_query.query_core:12 queries cache read from res_test_03_2018_10k_20220607_query_cache.pkl.
INFO:buildstock_query.query_core:12 queries cache is updated.


In [4]:
baseline_agg_query = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True,
                                          )



In [5]:
baseline_agg = my_run.execute("""
SELECT "res_test_03_2018_10k_20220607_baseline"."build_existing_model.geometry_building_type_recs" AS "geometry_building_type_recs", sum(1) AS "sample_count", sum("res_test_03_2018_10k_20220607_baseline"."build_existing_model.sample_weight") AS "units_count", sum("res_test_03_2018_10k_20220607_baseline"."report_simulation_output.fuel_use_electricity_total_m_btu" * "res_test_03_2018_10k_20220607_baseline"."build_existing_model.sample_weight") AS "fuel_use_electricity_total_m_btu" 
FROM "res_test_03_2018_10k_20220607_baseline" 
WHERE "res_test_03_2018_10k_20220607_baseline"."completed_status" = 'Success' GROUP BY 1
""")
baseline_agg

,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Multi-Family with 5+ Units,1189,1.583416e+07,4.099084e+08
1,Single-Family Attached,530,7.058119e+06,2.094629e+08
2,Single-Family Detached,7121,9.483182e+07,4.413546e+09
3,Mobile Home,703,9.361996e+06,4.506122e+08
4,Multi-Family with 2 - 4 Units,446,5.939474e+06,1.385683e+08


In [6]:
baseline_agg_query2 = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True)
print(baseline_agg_query2)

SELECT res_test_03_2018_10k_20220607_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(res_test_03_2018_10k_20220607_baseline."build_existing_model.sample_weight") AS units_count, sum(res_test_03_2018_10k_20220607_baseline."report_simulation_output.fuel_use_natural_gas_total_m_btu" * res_test_03_2018_10k_20220607_baseline."build_existing_model.sample_weight") AS fuel_use_natural_gas_total_m_btu 
FROM res_test_03_2018_10k_20220607_baseline 
WHERE res_test_03_2018_10k_20220607_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [7]:
bq = my_run.submit_batch_query([baseline_agg_query, baseline_agg_query2])
print(bq)
my_run.get_batch_query_report(bq)

1


{'Submitted': 0, 'Running': 0, 'Pending': 2, 'Completed': 0, 'Failed': 0}

INFO:buildstock_query.query_core:Submitted queries[0] (491c0a4e-4e5b-4a1a-a5fc-119c8edb501c)
INFO:buildstock_query.query_core:Submitted queries[1] (27ffe15f-6586-44f8-9897-2871446e3d66)


In [8]:
my_run.get_batch_query_report(bq)

{'Submitted': 2, 'Running': 0, 'Pending': 0, 'Completed': 2, 'Failed': 0}

In [9]:
result_df = my_run.get_batch_query_result(bq, combine=True)
result_df

INFO:buildstock_query.query_core:{'Submitted': 2, 'Running': 0, 'Pending': 0, 'Completed': 2, 'Failed': 0}
INFO:buildstock_query.query_core:Batch query completed. 
INFO:buildstock_query.query_core:Got result from Query [0] (491c0a4e-4e5b-4a1a-a5fc-119c8edb501c)
INFO:buildstock_query.query_core:Got result from Query [1] (27ffe15f-6586-44f8-9897-2871446e3d66)
INFO:buildstock_query.query_core:Concatenating the results.


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu,query_id,fuel_use_natural_gas_total_m_btu
0,Mobile Home,703,9.361996e+06,4.506122e+08,0,NaN
1,Multi-Family with 2 - 4 Units,446,5.939474e+06,1.385683e+08,0,NaN
2,Multi-Family with 5+ Units,1189,1.583416e+07,4.099084e+08,0,NaN
3,Single-Family Attached,530,7.058119e+06,2.094629e+08,0,NaN
4,Single-Family Detached,7121,9.483182e+07,4.413546e+09,0,NaN
0,Mobile Home,703,9.361996e+06,NaN,1,1.467286e+08
1,Multi-Family with 2 - 4 Units,446,5.939474e+06,NaN,1,3.638842e+08
2,Multi-Family with 5+ Units,1189,1.583416e+07,NaN,1,2.137026e+08
3,Single-Family Attached,530,7.058119e+06,NaN,1,3.356744e+08
4,Single-Family Detached,7121,9.483182e+07,NaN,1,6.115350e+09


In [13]:
query1 = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'], group_by=['state'],
                                              get_query_only=True)
query2 = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                              group_by=['state'],
                                              get_query_only=True)
exe_id1, future1 = my_run.execute(query1, run_async=True)
exe_id2, future2 = my_run.execute(query2, run_async=True)

In [14]:
exe_id1

'CACHED'

In [15]:
future1.as_pandas()

,state,sample_count,units_count,fuel_use_natural_gas_total_m_btu
0,AL,173,2.303877e+06,2.746831e+07
1,AR,100,1.331721e+06,3.919417e+07
2,AZ,217,2.889834e+06,3.297230e+07
3,CA,1051,1.399638e+07,4.360101e+08
4,CO,171,2.277242e+06,1.805907e+08
5,CT,115,1.531479e+06,5.755829e+07
6,DC,22,2.929785e+05,1.760066e+07
7,DE,32,4.261506e+05,2.158285e+07
8,FL,689,9.175555e+06,1.611552e+07
9,GA,308,4.101699e+06,1.151046e+08


In [13]:
my_run.get_query_output_location(query1)

's3://eulp/athena_query_results/a2fbc5ed-3ac6-4c03-8d01-7d11407510ac.csv'

In [14]:
my_run.get_query_status(query1)

'SUCCEEDED'

In [15]:
my_run.get_athena_query_result(query1)

INFO:aiobotocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


,state,sample_count,units_count,fuel_use_natural_gas_total_m_btu
0,PA,425,5.659812e+06,4.202414e+08
1,MO,196,2.610172e+06,1.730022e+08
2,NY,619,8.243350e+06,6.678062e+08
3,FL,689,9.175555e+06,1.611552e+07
4,WY,22,2.929785e+05,3.752161e+07
5,AZ,217,2.889834e+06,3.297230e+07
6,KY,143,1.904360e+06,6.927400e+07
7,AL,173,2.303877e+06,2.746831e+07
8,IA,100,1.331721e+06,1.534700e+08
9,OR,130,1.731237e+06,6.487875e+07


In [16]:
my_run.get_all_running_queries()

[]

In [17]:
my_run.stop_all_queries()

INFO:buildstock_query.query_core:Stopped 0 queries


In [18]:
my_run.stop_query(query1)

{'ResponseMetadata': {'RequestId': '40db05c5-7ebd-4892-8531-29e429107020',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 15 Nov 2022 18:49:56 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '40db05c5-7ebd-4892-8531-29e429107020'},
  'RetryAttempts': 0}}

In [19]:
my_run.delete_everything()

Will delete the following tables ['res_test_03_2018_10k_20220607_baseline', 'res_test_03_2018_10k_20220607_timeseries', 'res_test_03_2018_10k_20220607_upgrades'] and the s3:/euss/tests/res_test_03_2018_10k_20220607 folder
Abandoned the idea.
